In [1]:
import pyspark
sc = pyspark.SparkContext()

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel, LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
parsed_data = [LabeledPoint(0.0, [4.6,3.6,1.0,0.2]),
               LabeledPoint(0.0, [5.7,4.4,1.5,0.4]),
               LabeledPoint(1.0, [6.7,3.1,4.4,1.4]),
               LabeledPoint(0.0, [4.8,3.4,1.6,0.2]),
               LabeledPoint(2.0, [4.4,3.2,1.3,0.2])]     
#model = LogisticRegressionWithSGD.train(sc.parallelize(parsed_data)) # gives error:
# org.apache.spark.SparkException: Input validation failed.
model = LogisticRegressionWithLBFGS.train(sc.parallelize(parsed_data), numClasses=3)  # works OK

In [16]:
import sqlite3
import datetime
import time
from time import strptime
from datetime import datetime

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest


bd = sqlite3.connect('velib3.db')
c = bd.cursor()

def feedData(jour, mois, annee, num_station):

    #select * from station_dyn where num = 10114  and db_write_time >= ('16/03/2017') and db_write_time < ('17/03/2017') order by last_update

    enregistrements=[]

    week_day = datetime(annee, mois, jour).timetuple()[6]
    if week_day not in (5,6):
        hist=[-1,-2,-3,-7,-14]
    else:
        hist=[-7,-8,-14,-15]
        if week_day == 6:
            hist.append(-1)

    for i in hist:
        if datetime(annee, mois, jour+i).timetuple()[6] in (5,6):
            hist.append(i-2)
            continue
        clause ="last_update_date >= (\'"+str(datetime(annee, mois, jour+i))[:10]+"\') and last_update_date < (\'"+ str(datetime(annee, mois, jour+i+1))[:10]+"\')"
        c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where "+clause+" and num = "+str(num_station)+" order by last_update")
        enregistrements.extend(c.fetchall())
        
    #print enregistrements
    #print len(enregistrements)
    
    return enregistrements

#test = feedData(28, 3, 2017, 15003)
#print test



c.execute("select DISTINCT num from station_dyn" )

list_station=[]

for row in c:
    list_station.append(row[0]) 

"""
for h in range(0,24):
    for m in range(0,51,10):
        #print ("0" if len(str(h))==1 else "")+str(h)+":"+str(m)+("0" if m==0 else "") #144 intervalles par jour
        pass"""

data=[]
interval_pred=600*3


for station in range(15003,15004):#0,1):#len(list_station)):
    list_row=feedData(28, 3, 2017, 15003)
    
    #c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where num = "+str(15003)+" order by last_update")
    #c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where num = "+str(list_station[station])+" order by last_update")
    
    #list_row=c.fetchall()
    
    """i=0
    while list_row!=[]:
        print list_row.pop(0)
        i+=1
        print str(i)+(" vide" if list_row==[] else " non vide")
    print len(list_row)
    break"""
    
    #print list_row[-1]
    #print time.strftime("%d/%m/%Y-%H:%M:%S", time.gmtime((int(list_row[-1][1])/1000/600)*600))
    #print time.strftime("%d/%m/%Y-%H:%M:%S", time.gmtime((int(list_row[0][1])/1000/600)*600+600))
    
    row=1
    boucle=0
    in_flow=0
    out_flow=0
    date_interval = (int(list_row[0][1])/1000/interval_pred)*interval_pred+interval_pred
    #for date in range((int(list_row[0][1])/1000/600)*600+600,(int(list_row[-1][1])/1000/600)*600+1,600):
        #print datetime.datetime.fromtimestamp(date).strftime('%H:%M'), datetime.datetime.fromtimestamp(date).strftime('%d-%m')
        
        #implémenter les filtres
    prec_row=list_row[0]
    current_row=list_row.pop(0) #Erreur si la liste est vide
    while list_row!=[]:
        current_row=list_row.pop(0)
        row+=1
        
        flow=current_row[0]-prec_row[0]
        
        if int(current_row[1])/1000<date_interval:
            boucle+=1
            in_flow += flow if flow>0 else 0
            out_flow += flow if flow<0 else 0
        else: #in_flow : nb_velo_dispo, week-day, heure_normalisée
            data.append(LabeledPoint(float(in_flow),[float(current_row[0]),time.gmtime(int(current_row[1])/1000)[6],int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*6+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/10]))
            print [LabeledPoint(float(in_flow),\
                   [time.gmtime(int(current_row[1])/1000)[6],\
                   int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*6+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/10])]
            while int(current_row[1])/1000>=date_interval:
                date_interval+=interval_pred
            boucle=1
            in_flow = flow if flow>0 else 0
            out_flow = flow if flow<0 else 0
        
        print boucle, row, current_row, time.strftime("%H:%M:%S", time.gmtime((int(current_row[1])/1000+3600)-(date_interval+3600-interval_pred)))
        print "intervalle : ["+str(time.strftime("%H:%M", time.gmtime(date_interval+3600-interval_pred)))+","+str(time.strftime("%H:%M", time.gmtime(date_interval+3600)))+"["+" -  n° : "+str(int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/30) #doit être modifié à la main si changemenr d'intervalle
        
        print "in flow : "+str(in_flow)+" - out flow : "+str(out_flow)
        
        prec_row=current_row




c.close()

2055
1 2 (0, u'1490566818000', u'2017-03-27 00:20:18') 00:20:18
intervalle : [23:00,23:30[ -  n° : 46
in flow : 0 - out flow : 0
2 3 (0, u'1490566908000', u'2017-03-27 00:21:48') 00:21:48
intervalle : [23:00,23:30[ -  n° : 46
in flow : 0 - out flow : 0
3 4 (0, u'1490566975000', u'2017-03-27 00:22:55') 00:22:55
intervalle : [23:00,23:30[ -  n° : 46
in flow : 0 - out flow : 0
4 5 (0, u'1490567050000', u'2017-03-27 00:24:10') 00:24:10
intervalle : [23:00,23:30[ -  n° : 46
in flow : 0 - out flow : 0
[LabeledPoint(0.0, [6.0,138.0])]
1 6 (0, u'1490567654000', u'2017-03-27 00:34:14') 00:04:14
intervalle : [23:30,00:00[ -  n° : 47
in flow : 0 - out flow : 0
2 7 (0, u'1490568258000', u'2017-03-27 00:44:18') 00:14:18
intervalle : [23:30,00:00[ -  n° : 47
in flow : 0 - out flow : 0
3 8 (0, u'1490568862000', u'2017-03-27 00:54:22') 00:24:22
intervalle : [23:30,00:00[ -  n° : 47
in flow : 0 - out flow : 0
[LabeledPoint(0.0, [6.0,141.0])]
1 9 (0, u'1490569466000', u'2017-03-27 01:04:26') 00:04:26
in

In [17]:
print len(data)
print "********"
print data

46
********
[LabeledPoint(0.0, [0.0,6.0,138.0]), LabeledPoint(0.0, [0.0,6.0,141.0]), LabeledPoint(0.0, [0.0,6.0,0.0]), LabeledPoint(1.0, [1.0,0.0,3.0]), LabeledPoint(0.0, [0.0,0.0,6.0]), LabeledPoint(0.0, [0.0,0.0,9.0]), LabeledPoint(0.0, [0.0,0.0,15.0]), LabeledPoint(0.0, [0.0,0.0,18.0]), LabeledPoint(0.0, [0.0,0.0,21.0]), LabeledPoint(0.0, [0.0,0.0,24.0]), LabeledPoint(0.0, [0.0,0.0,27.0]), LabeledPoint(0.0, [0.0,0.0,30.0]), LabeledPoint(0.0, [0.0,0.0,33.0]), LabeledPoint(3.0, [3.0,0.0,36.0]), LabeledPoint(5.0, [4.0,0.0,39.0]), LabeledPoint(9.0, [6.0,0.0,42.0]), LabeledPoint(9.0, [4.0,0.0,45.0]), LabeledPoint(5.0, [2.0,0.0,48.0]), LabeledPoint(3.0, [1.0,0.0,51.0]), LabeledPoint(0.0, [1.0,0.0,54.0]), LabeledPoint(2.0, [1.0,0.0,57.0]), LabeledPoint(0.0, [1.0,0.0,60.0]), LabeledPoint(2.0, [1.0,0.0,63.0]), LabeledPoint(1.0, [2.0,0.0,66.0]), LabeledPoint(4.0, [1.0,0.0,69.0]), LabeledPoint(4.0, [0.0,0.0,72.0]), LabeledPoint(5.0, [2.0,0.0,75.0]), LabeledPoint(1.0, [1.0,0.0,78.0]), LabeledPo